In [1]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf

import PyNetwork
import pyopencl as cl
import pyopencl.array as cl_array

In [2]:
platform = cl.get_platforms()
devices = platform[0].get_devices()
context = cl.Context(devices)
queue = cl.CommandQueue(context)

In [3]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

shape = (28, 28)
x_train = x_train.reshape(-1, *shape) / 255
x_test = x_test.reshape(-1, *shape) / 255

labels = np.eye(10)

y_train = labels[y_train.flatten()]
y_test = labels[y_test.flatten()]

# Transform x_train and y_train into C-contiguous
x_train = np.ascontiguousarray(x_train)
y_train = np.ascontiguousarray(y_train)
x_test = np.ascontiguousarray(x_test)
y_test = np.ascontiguousarray(y_test)

# Send data to device
x_train_gpu = cl_array.to_device(queue, x_train)
y_train_gpu = cl_array.to_device(queue, y_train)
x_test_gpu = cl_array.to_device(queue, x_test)
y_test_gpu = cl_array.to_device(queue, y_test)

# Large Learning Rate

In [4]:
model = PyNetwork.Sequential()

model.add(PyNetwork.layers.Input((28, 28)))
model.add(PyNetwork.layers.Flatten())
model.add(PyNetwork.layers.Dense(100, activation_function='relu', l2=0.01, l1=0.0))
# model.add(PyNetwork.layers.BatchNorm())
model.add(PyNetwork.layers.Dense(10, activation_function='relu', l2=0.0, l1=0.0))

optimizer = PyNetwork.optimizers.RMSprop(learning_rate=0.0005)
model.build(context, queue, loss_function='cross_entropy', optimizer=optimizer, metrics='accuracy')

In [5]:
%%time
model.train(x_train_gpu, y_train_gpu, epochs=10, batch_size=128, verbose=True)

Training on 60000 samples
Epoch 1/10
cross_entropy: 20.7233 - accuracy: 0.5000
Training on 60000 samples
Epoch 2/10
cross_entropy: 20.7233 - accuracy: 0.5000
Training on 60000 samples
Epoch 3/10
cross_entropy: 20.7233 - accuracy: 0.5000
Training on 60000 samples
Epoch 4/10
cross_entropy: 20.7233 - accuracy: 0.5000
Training on 60000 samples
Epoch 5/10
cross_entropy: 20.7233 - accuracy: 0.5000
Training on 60000 samples
Epoch 6/10
cross_entropy: 20.7233 - accuracy: 0.5000
Training on 60000 samples
Epoch 7/10
cross_entropy: 20.7233 - accuracy: 0.5000
Training on 60000 samples
Epoch 8/10
cross_entropy: 20.7233 - accuracy: 0.5000
Training on 60000 samples
Epoch 9/10
cross_entropy: 20.7233 - accuracy: 0.5000
Training on 60000 samples
Epoch 10/10
cross_entropy: 20.7233 - accuracy: 0.5000
CPU times: total: 1min 28s
Wall time: 3min 1s


In [6]:
model.evaluate(x_test_gpu, y_test_gpu)

'cross_entropy: 20.7233 - accuracy: -0.0120'

# Smaller Learning Rate

In [7]:
model = PyNetwork.Sequential()

model.add(PyNetwork.layers.Input((28, 28)))
model.add(PyNetwork.layers.Flatten())
model.add(PyNetwork.layers.Dense(100, activation_function='relu', l2=0.01, l1=0.0))
model.add(PyNetwork.layers.BatchNorm())
model.add(PyNetwork.layers.Dense(10, activation_function='softmax', l2=0.0, l1=0.0))

optimizer = PyNetwork.optimizers.RMSprop(learning_rate=0.0001)
model.build(context, queue, loss_function='cross_entropy', optimizer=optimizer, metrics='accuracy')

TypeError: Sequential.build() missing 2 required positional arguments: 'context' and 'queue'

In [ ]:
%%time
model.train(x_train, y_train, epochs=10, batch_size=128, verbose=True)

Training on 60000 samples
Epoch 1/10
cross_entropy: 1.6821 - accuracy: 0.5391
Training on 60000 samples
Epoch 2/10
cross_entropy: 1.3679 - accuracy: 0.6484
Training on 60000 samples
Epoch 3/10
cross_entropy: 1.1780 - accuracy: 0.7266
Training on 60000 samples
Epoch 4/10
cross_entropy: 0.9567 - accuracy: 0.7734
Training on 60000 samples
Epoch 5/10
cross_entropy: 0.8732 - accuracy: 0.7422
Training on 60000 samples
Epoch 6/10
cross_entropy: 0.9316 - accuracy: 0.7266
Training on 60000 samples
Epoch 7/10
cross_entropy: 0.7509 - accuracy: 0.7422
Training on 60000 samples
Epoch 8/10
cross_entropy: 0.7198 - accuracy: 0.7578
Training on 60000 samples
Epoch 9/10
cross_entropy: 0.5410 - accuracy: 0.8125
Training on 60000 samples
Epoch 10/10
cross_entropy: 0.5575 - accuracy: 0.8281
CPU times: user 2min 55s, sys: 9.12 s, total: 3min 5s
Wall time: 20.7 s


In [ ]:
model.evaluate(x_test, y_test)

'cross_entropy: 0.6207 - accuracy: 0.8025'